In [26]:
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [55]:
data = arff.loadarff('./Data/cm1.arff.txt')
df = pd.DataFrame(data[0])
df['defects'] = df['defects'].apply(lambda x: str(x)[1:]) #removing 'b' from classes
numberOfColumns = len(df.columns) #qtd of columns with defects column
justClass = df[['defects']].values #only the classes values
justData = df.drop(['defects'], axis=1) #removing the classes column for normalization
valuesOfColums = justData.columns.values #get attributes names
justDataValues = justData.values #get data as numpy array

In [58]:
min_max_scaler = preprocessing.MinMaxScaler()
normalized_data = min_max_scaler.fit_transform(justDataValues)
df = pd.DataFrame(normalized_data)
df.columns = valuesOfColums
df.insert(loc=numberOfColumns-1, column='defects', value=justClass)